In [11]:
import py3Dmol
from rdkit import Chem
import ast
import os
import os.path as osp
from preprocessing_forked_ET_Flow import load_pkl, load_json


def draw(from_conf, path, mol_id, conf_id=0, partition="qm9"):
    if from_conf:
        mols = load_json(osp.join(path, f"summary_{partition}.json"))
        for id, (_, mol_dict) in enumerate(mols.items()):
            if id < mol_id:
                continue
            mol_pickle = load_pkl(os.path.join(path, mol_dict["pickle_path"]))
            break
        confs = mol_pickle["conformers"]
        
        if conf_id >= len(confs):
            print(f"Conformer ID {conf_id} out of range for molecule ID {mol_id}.")
            return
        
        conf = confs[conf_id]
        mol = conf["rd_mol"]
    else:
        suppl = Chem.SDMolSupplier(path, removeHs=False)
        mol = suppl[mol_id] if suppl else None
    
    # Try to get the _smilesAtomOutputOrder property; if not found, use the atom indices.
    try:
        atom_order = ast.literal_eval(mol.GetProp('_smilesAtomOutputOrder'))
    except Exception:
        atom_order = list(range(mol.GetNumAtoms()))
    
    # Build a mapping from SDF atom index to its order value.
    order_labels = {}
    for idx in range(mol.GetNumAtoms()):
        if idx in atom_order:
            order_labels[idx] = atom_order.index(idx)
        else:
            order_labels[idx] = idx

    conf = mol.GetConformer()
    mol_block = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=400, height=400)
    viewer.addModel(mol_block, "sdf")
    viewer.setStyle({"stick": {}})
    
    # Label each atom with its symbol and the computed order.
    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        pos = conf.GetAtomPosition(idx)
        label = f"{atom.GetSymbol()}{order_labels[idx]}"
        viewer.addLabel(label,
                        {"position": {"x": pos.x, "y": pos.y, "z": pos.z},
                         "backgroundColor": "gray",
                         "fontColor": "white"})
    
    viewer.zoomTo()
    viewer.show()

In [12]:
draw(True, "/mnt/sxtn2/chem/GEOM_data/rdkit_folder", 22165, 0)
draw(False, "/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/special_mol1.sdf", 0)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.